<a href="https://colab.research.google.com/github/Kridr/HW_PB_HSE/blob/main/HW7_PB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from Bio import SeqIO
import pandas as pd
from collections import defaultdict

In [3]:
gb_record = SeqIO.read(open("sequence.gb", "r"), "genbank")

In [4]:
d = defaultdict(list)
for gb_feature in gb_record.features:
    if gb_feature.type == 'gene':
        d['start'].append(gb_feature.location.start)
        d['end'].append(gb_feature.location.end)

df_genbank = pd.DataFrame.from_dict(d)
df_genbank.head(10)

,start,end
0,84,597
1,796,2835
2,2350,4814
3,5085,5808
4,5852,6518
5,6246,9593
6,9588,13712
7,9715,11446
8,13801,17450
9,17253,19136


In [5]:
df_genemark = pd.read_csv('genemark.txt', sep='\s+', index_col='Gene', skiprows=9)
df_genemark.LeftEnd[1] = 1
df_genemark.LeftEnd = df_genemark.LeftEnd.astype(int)
df_genemark.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Strand,LeftEnd,RightEnd,Gene_Length,Class
Gene,,,,,
1,+,1,597,597,1
2,+,828,2153,1326,1
3,-,2836,4752,1917,1
4,+,5104,5679,576,1
5,+,5888,6307,420,1
6,-,6949,7326,378,1
7,-,8006,8578,573,1
8,-,8675,9544,870,1
9,+,9731,11119,1389,1


In [6]:
len(df_genbank), len(df_genemark)

(57, 62)

Число (и % от общего числа в исходном файле) генов, которые аннотированы в БД GenBank и для которых предсказание обоих концов гена с помощью Genemark точно такое же.

Здесь просто перебираем гены из GenBank и GeneMark и ищем совпадения (совпадения как начала, так и концов)

In [7]:
s = 0
for i in df_genbank.index:
    for j in df_genemark.index:
        if df_genbank.start[i] == df_genemark.LeftEnd[j] and df_genbank.end[i] == df_genemark.RightEnd[j]:
            s += 1
            break

print(s, round(s / len(df_genbank) * 100, 2))

0 0.0


Число (и %) генов, для которых аннотация только N-конца белка не совпадает с аннотацией (не забывайте смотреть на ориентацию гена)


Здесь для каждого начала гена из GenBank проверяем есть ли он в начале какого-нибудь гена из GeneMark

In [8]:
s = 0
for e in df_genbank.start:
    if e not in list(df_genemark.LeftEnd):
        s += 1

print(s, round(s / len(df_genbank) * 100, 2))

57 100.0


Число (и %) генов, для которых аннотация только С-конца белка не совпадает с аннотацией Genemark.


Здесь для каждого конца гена из GenBank проверяем есть ли он в конце какого-нибудь гена из GeneMark

In [9]:
s = 0
for e in df_genbank.end:
    if e not in list(df_genemark.RightEnd):
        s += 1

print(s, round(s / len(df_genbank) * 100, 2))

49 85.96


Число (и %) генов, для которых аннотация обоих концов не совпадает с аннотацией Genemark.

Здесь итерируемся по всем генам из GenBank и GenMark. Для каждого гена из GenBank проверяем отсутствие совпадения (то есть его начало и конец не совпадают с началом и концом какого-либо гена из GenMark)  

In [10]:
s = 0
for i in df_genbank.index:
    cond = False
    for j in df_genemark.index:
        if df_genbank.start[i] == df_genemark.LeftEnd[j] and df_genbank.end[i] == df_genemark.RightEnd[j]:
            cond = True
            break
    if not cond:
        s += 1

print(s, round(s / len(df_genbank) * 100, 2))

57 100.0


Число предсказанных генов не пересекающихся с генами из записи

Здесь опять же итерируемся как по генам из GeneMark, так и по генам из GenBank. Если для очередного гена из GeneMark нет ни одного гена из GenBank, с которым он пересекатеся, то инкрементируем счетчик. В данном случае пересечение - это либо нахождение начала конца одного гена между началом и концом другого гена, либо нахождение конца одного одного гена между началом и концом другого гена,
либо вложенность

In [11]:
s = 0
for i in df_genemark.index:
    cond = False
    for j in df_genbank.index:
        inters1 = (df_genemark.LeftEnd[i] >= df_genbank.start[j] and df_genemark.LeftEnd[i] <= df_genbank.end[j])
        inters2 = (df_genemark.RightEnd[i] >= df_genbank.start[j] and df_genemark.RightEnd[i] <= df_genbank.end[j])
        inters3 = (df_genemark.LeftEnd[i] <= df_genbank.start[j] and df_genemark.RightEnd[i] >= df_genbank.end[j])
        if inters1 or inters2 or inters3:
            cond = True
            break
    if not cond:
        s += 1

print(s, round(s / len(df_genemark) * 100, 2))

0 0.0


Число генов в записи не пересекающихся с предсказаниями

Аналогично предыдущему пункту, но GenBank и GeneMark меняются местами

In [12]:
s = 0
for i in df_genbank.index:
    cond = False
    for j in df_genemark.index:
        inters1 = df_genbank.start[i] >= df_genemark.LeftEnd[j] and df_genbank.start[i] <= df_genemark.RightEnd[j]
        inters2 = df_genbank.end[i] >= df_genemark.LeftEnd[j] and df_genbank.end[i] <= df_genemark.RightEnd[j]
        inters3 = df_genbank.start[i] <= df_genemark.LeftEnd[j] and df_genbank.end[i] >= df_genemark.RightEnd[j]
        if inters1 or inters2 or inters3:
            cond = True
            break
    if not cond:
        s += 1

print(s, round(s / len(df_genbank) * 100, 2))

8 14.04
